In [1]:
import gurobipy as gp
from gurobipy import GRB

# Create model
m = gp.Model("nonconvex_qp")

# Add variables
x = m.addVar(lb=-GRB.INFINITY, name="x")
y = m.addVar(lb=-GRB.INFINITY, name="y")

# Non-convex quadratic objective: minimize x * y
m.setObjective(x * y, GRB.MINIMIZE)

# Add constraints
m.addConstr(x + y == 1, "c0")
m.addConstr(x >= -2, "c1")
m.addConstr(y >= -2, "c2")

# Enable nonconvex optimization
m.setParam("NonConvex", 2)

# Optimize
m.optimize()

# Print solution
if m.status == GRB.OPTIMAL:
    print(f"Optimal objective: {m.objVal}")
    print(f"x = {x.X}, y = {y.X}")


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2709591
Academic license 2709591 - for non-commercial use only - registered to pi___@berkeley.edu
Set parameter NonConvex to value 2
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Zorin OS 17.3")

CPU model: AMD Ryzen 9 7950X3D 16-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads

Non-default parameters:
NonConvex  2

Academic license 2709591 - for non-commercial use only - registered to pi___@berkeley.edu
Optimize a model with 3 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x34ff4dc7
Model has 1 quadratic objective term
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 2 rows and 0 columns

Continuous model is non-convex -- solving as a MIP

Pr

In [ ]:

import numpy as np

# Covariance matrix and expected returns
Sigma = np.array([
    [0.1, 0.02, 0.04, 0.01],
    [0.02, 0.08, 0.02, 0.03],
    [0.04, 0.02, 0.12, 0.02],
    [0.01, 0.03, 0.02, 0.07]
])
mu = np.array([0.10, 0.12, 0.14, 0.09])
Rtarget = 0.115

# Create model
m = gp.Model("nonconvex_portfolio")

# Add variables for weights
w = m.addVars(4, lb=0, ub=1, name="w")

# Objective: minimize variance = wᵀ Σ w
quad_expr = gp.QuadExpr()
for i in range(4):
    for j in range(4):
        quad_expr += w[i] * Sigma[i, j] * w[j]
m.setObjective(quad_expr, GRB.MINIMIZE)

# Constraints
m.addConstr(gp.quicksum(w[i] for i in range(4)) == 1, "budget")

# Non-convex quadratic return constraint
# (muᵀ w - Rtarget)(muᵀ w - (Rtarget+0.01)) ≤ 0
ret_expr = gp.quicksum(mu[i] * w[i] for i in range(4))
m.addQConstr((ret_expr - Rtarget) * (ret_expr - (Rtarget + 0.01)) <= 0, "nonconvex_return")

# Enable nonconvex solver
m.setParam("NonConvex", 2)

# Optimize
m.optimize()

# Output solution
if m.status == GRB.OPTIMAL:
    print("Optimal variance:", m.objVal)
    for i in
